This notebook performs profile annotation.
The platemap is mapped back to the profile to retain the sample metadata.


In [1]:
import argparse
import pathlib

import pandas as pd

# Get the current working directory
cwd = pathlib.Path.cwd()

if (cwd / ".git").is_dir():
    root_dir = cwd

else:
    root_dir = None
    for parent in cwd.parents:
        if (parent / ".git").is_dir():
            root_dir = parent
            break

# Check if a Git root directory was found
if root_dir is None:
    raise FileNotFoundError("No Git root directory found.")
try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False

In [2]:
if not in_notebook:
    argparser = argparse.ArgumentParser()
    argparser.add_argument(
        "--patient",
        type=str,
        required=True,
        help="Patient ID to process, e.g. 'P01'",
    )
    args = argparser.parse_args()
    patient = args.patient

else:
    patient = "NF0014"

In [3]:
def annotate_profiles(
    profile_df: pd.DataFrame, platemap_df: pd.DataFrame, patient: str
) -> pd.DataFrame:
    """
    Annotate profiles with treatment, dose, and unit information from the platemap.

        Parameters
        ----------
        profile_df : pd.DataFrame
            Profile DataFrame containing image_set information.
            Could be either single-cell or organoid profiles.
        platemap_df : pd.DataFrame
            Platmap DataFrame containing well_position, treatment, dose, and unit.
        patient : str
            Patient ID to annotate the profiles with.

        Returns
        -------
        pd.DataFrame
            Annotated profile DataFrame with additional columns for treatment, dose, and unit.
    """
    drug_information = pd.read_csv(
        pathlib.Path(
            f"{root_dir}/4.processing_image_based_profiles/data/drugs/drug_information.csv"
        )
    )
    profile_df["Well"] = profile_df["image_set"].str.split("-").str[0]
    profile_df.insert(2, "Well", profile_df.pop("Well"))
    profile_df = pd.merge(
        profile_df,
        platemap_df[["well_position", "treatment", "dose", "unit"]],
        left_on="Well",
        right_on="well_position",
        how="left",
    ).drop(columns=["well_position"])
    profile_df = profile_df.merge(
        drug_information, how="left", left_on="treatment", right_on="Treatment"
    )
    profile_df.drop(columns=["Treatment"], inplace=True)
    for col in ["treatment", "dose", "unit"]:
        profile_df.insert(1, col, profile_df.pop(col))
    profile_df.insert(0, "patient", patient)
    return profile_df

In [4]:
# pathing

sc_merged_path = pathlib.Path(
    f"{root_dir}/data/{patient}/image_based_profiles/1.combined_profiles/sc.parquet"
).resolve(strict=True)
organoid_merged_path = pathlib.Path(
    f"{root_dir}/data/{patient}/image_based_profiles/1.combined_profiles/organoid.parquet"
).resolve(strict=True)

platemap_path = pathlib.Path(
    f"{root_dir}/data/{patient}/platemap/platemap.csv"
).resolve(strict=True)

# output path
sc_annotated_output_path = pathlib.Path(
    f"{root_dir}/data/{patient}/image_based_profiles/2.annotated_profiles/sc_anno.parquet"
).resolve()
organoid_annotated_output_path = pathlib.Path(
    f"{root_dir}/data/{patient}/image_based_profiles/2.annotated_profiles/organoid_anno.parquet"
).resolve()

organoid_annotated_output_path.parent.mkdir(parents=True, exist_ok=True)

In [5]:
# read data
sc_merged = pd.read_parquet(sc_merged_path)
organoid_merged = pd.read_parquet(organoid_merged_path)
# read platemap
platemap = pd.read_csv(platemap_path)
platemap.head()

,WellRow,WellCol,well_position,treatment,dose,unit
0,C,2,C2,Staurosporine,10,nM
1,D,2,D2,Digoxin,1,uM
2,E,2,E2,Digoxin,1,uM
3,F,2,F2,Onalespib,1,uM
4,G,2,G2,Staurosporine,10,nM


In [6]:
sc_merged = annotate_profiles(sc_merged, platemap, patient)
organoid_merged = annotate_profiles(organoid_merged, platemap, patient)

In [7]:
sc_merged.head()

,patient,object_id,unit,dose,treatment,image_set,Well,parent_organoid,Colocalization_Nuclei_AGP.BF_MEAN.CORRELATION.COEFF,Colocalization_Nuclei_AGP.BF_MEDIAN.CORRELATION.COEFF,...,Area.Size.Shape_Cytoplasm_MAX.Y,Area.Size.Shape_Cytoplasm_MIN.Z,Area.Size.Shape_Cytoplasm_MAX.Z,Area.Size.Shape_Cytoplasm_EXTENT,Area.Size.Shape_Cytoplasm_EULER.NUMBER,Area.Size.Shape_Cytoplasm_EQUIVALENT.DIAMETER,Area.Size.Shape_Cytoplasm_SURFACE.AREA,Target,Class,Therapeutic Categories
0,NF0014,25,uM,10,Mirdametinib,G8-1,G8,40,-0.039767,-0.039767,...,909.0,0.0,29.0,0.100018,-44.0,86.478996,8537.206055,MEK1/2 inhibitor,Small Molecule,Kinase Inhibitor
1,NF0014,51,uM,10,Mirdametinib,G8-1,G8,40,0.028712,0.028712,...,916.0,0.0,41.0,0.131350,-54.0,90.847000,7764.473145,MEK1/2 inhibitor,Small Molecule,Kinase Inhibitor
2,NF0014,76,uM,10,Mirdametinib,G8-1,G8,40,-0.027481,-0.027481,...,762.0,0.0,41.0,0.162104,-36.0,81.312607,3883.599609,MEK1/2 inhibitor,Small Molecule,Kinase Inhibitor
3,NF0014,102,uM,10,Mirdametinib,G8-1,G8,40,-0.012504,-0.012504,...,872.0,0.0,39.0,0.159934,-37.0,62.487492,1958.485352,MEK1/2 inhibitor,Small Molecule,Kinase Inhibitor
4,NF0014,127,uM,10,Mirdametinib,G8-1,G8,40,0.063136,0.063136,...,786.0,0.0,41.0,0.211394,-55.0,73.653473,3029.156738,MEK1/2 inhibitor,Small Molecule,Kinase Inhibitor


In [8]:
organoid_merged.head()

,patient,object_id,unit,dose,treatment,image_set,Well,single_cell_count,Colocalization_Organoid_AGP.BF_MEAN.CORRELATION.COEFF,Colocalization_Organoid_AGP.BF_MEDIAN.CORRELATION.COEFF,...,Area.Size.Shape_Organoid_MAX.Y,Area.Size.Shape_Organoid_MIN.Z,Area.Size.Shape_Organoid_MAX.Z,Area.Size.Shape_Organoid_EXTENT,Area.Size.Shape_Organoid_EULER.NUMBER,Area.Size.Shape_Organoid_EQUIVALENT.DIAMETER,Area.Size.Shape_Organoid_SURFACE.AREA,Target,Class,Therapeutic Categories
0,NF0014,40,uM,10,Mirdametinib,G8-1,G8,8.0,-0.047442,-0.047442,...,1162.0,0.0,41.0,0.267881,1.0,230.547821,9698.893555,MEK1/2 inhibitor,Small Molecule,Kinase Inhibitor
1,NF0014,24,uM,1,Fimepinostat,D5-1,D5,5.0,0.034538,0.034538,...,1518.0,0.0,11.0,0.144968,5.0,192.377014,33464.886719,PI3K and HDAC inhibitor,Small Molecule,Investigational
2,NF0014,85,uM,1,Mirdametinib,F8-1,F8,9.0,0.018870,0.018870,...,1537.0,0.0,23.0,0.418138,14.0,336.359833,24271.376953,MEK1/2 inhibitor,Small Molecule,Kinase Inhibitor
3,NF0014,18,nM,10,Staurosporine,G11-1,G11,1.0,0.005859,0.005859,...,1085.0,0.0,19.0,0.557427,1.0,173.222839,3804.474365,Apoptosis induction,Small Molecule,Experimental
4,NF0014,24,uM,1,Binimetinib,G7-1,G7,7.0,-0.022475,-0.022475,...,1227.0,0.0,25.0,0.462178,1.0,229.673218,6281.933594,MEK1/2 inhibitor,Small Molecule,Kinase Inhibitor


In [9]:
# save annotated profiles
sc_merged.to_parquet(sc_annotated_output_path, index=False)
organoid_merged.to_parquet(organoid_annotated_output_path, index=False)